In [1]:
!pip install transformers
!pip install torch
!pip install nltk

In [9]:
import nltk
from nltk.corpus import stopwords


try:
    stop_words = set(stopwords.words('english'))
except LookupError:
    nltk.download('stopwords')
    stop_words = set(stopwords.words('english'))


In [13]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer


tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

def generate_text(prompt, max_length=150):
    inputs = tokenizer.encode(prompt, return_tensors='pt')
    outputs = model.generate(inputs, max_length=max_length, num_return_sequences=1)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [15]:
import random

def generate_mcq(chat_history):
    
    context = preprocess_chat(chat_history)
    
    
    prompt = f"Generate a multiple-choice question from the following context: {context}\nQuestion:"
    
    question = generate_text(prompt, max_length=50)
    
    
    correct_prompt = f"Generate the correct answer for the question: {question}\nAnswer:"
    correct_answer = generate_text(correct_prompt, max_length=20)
    
    
    distractors = []
    for i in range(3):
        distractor_prompt = f"Generate a plausible but incorrect answer for the question: {question}\nIncorrect answer:"
        distractors.append(generate_text(distractor_prompt, max_length=20))
    
    return {
        "question": question.strip(),
        "correct_answer": correct_answer.strip(),
        "distractors": [d.strip() for d in distractors]
    }


In [17]:
def generate_article(chat_history):
   
    context = preprocess_chat(chat_history)
    
    
    prompt = f"Generate a concise 200-300 word article from the following conversation: {context}\nArticle:"
    
    
    article = generate_text(prompt, max_length=300)
    
    return article.strip()

In [ ]:
def main():
    
    print("Enter the chat history (type 'done' when finished):")
    chat_history = []
    
    while True:
        line = input()
        if line.lower() == 'done':
            break
        chat_history.append(line)
    
    if not chat_history:
        print("No chat history provided. Exiting...")
        return
    
    
    mcq = generate_mcq(chat_history)
    print("\nMultiple Choice Question:")
    print(mcq["question"])
    print(f"A) {mcq['correct_answer']}")
    for i, distractor in enumerate(mcq['distractors'], start=1):
        print(f"{chr(65+i)}) {distractor}")
    
    
    article = generate_article(chat_history)
    print("\nGenerated Article:")
    print(article)

if __name__ == "__main__":
    main()


Enter the chat history (type 'done' when finished):
